In [ ]:
!pip install scikit-learn nltk

In [7]:
!pip install evaluate rouge

In [1]:
!pip install transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 8.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [8]:
import nltk
import numpy as np
from nltk.corpus import reuters
# разделение данных на тренировочный, валидационный и тестовый наборы
from sklearn.model_selection import train_test_split
# очистка данных
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
# преобразование текста в числовое представление
from sklearn.feature_extraction.text import CountVectorizer
# обработка меток
from sklearn.preprocessing import MultiLabelBinarizer
# bert
import torch
from transformers import BertTokenizer
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import accuracy_score, f1_score

from nltk.translate.bleu_score import sentence_bleu
from rouge import Rouge

# оптимизаторы и планировщики
from torch.optim import AdamW, SGD, RMSprop
from transformers import get_linear_schedule_with_warmup, get_cosine_schedule_with_warmup

import pandas as pd

In [49]:
# загрузка данных
nltk.download('reuters')
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt_tab')

# получение данных
doc_ids = reuters.fileids()
train_ids = [doc_id for doc_id in doc_ids if doc_id.startswith('training')]
test_ids = [doc_id for doc_id in doc_ids if doc_id.startswith('test')]

train_data = [reuters.raw(doc_id) for doc_id in train_ids]
train_labels = [reuters.categories(doc_id) for doc_id in train_ids]

test_data = [reuters.raw(doc_id) for doc_id in test_ids]
test_labels = [reuters.categories(doc_id) for doc_id in test_ids]

[nltk_data] Downloading package reuters to /root/nltk_data...
[nltk_data]   Package reuters is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [50]:
# разделение тренировочных данных на тренировочный и валидационный наборы
train_data, val_data, train_labels, val_labels = train_test_split(
    train_data, train_labels, test_size=0.2, random_state=42
)

# вывод размеров наборов данных
print("тренировочные данные:", len(train_data))
print("валидационные данные:", len(val_data))
print("тестовые данные:", len(test_data))

тренировочные данные: 80
валидационные данные: 20
тестовые данные: 15


In [51]:
# инициализация стоп-слов и лемматизатора
stop_words = set(stopwords.words('english'))  # для английского языка
lemmatizer = WordNetLemmatizer()

# функция для очистки текста
def clean_text(text):
    # удаление пунктуации
    text = re.sub(r'[^\w\s]', '', text)

    # приведение к нижнему регистру
    text = text.lower()

    # токенизация
    words = word_tokenize(text)

    # удаление стоп-слов
    words = [word for word in words if word not in stop_words]

    # лемматизация
    words = [lemmatizer.lemmatize(word) for word in words]

    return ' '.join(words)

# применение очистки ко всем данным
train_data_cleaned = [clean_text(text) for text in train_data]
val_data_cleaned = [clean_text(text) for text in val_data]
test_data_cleaned = [clean_text(text) for text in test_data]
# пример очищенных данных
print("очищенный тренировочный текст:", train_data_cleaned[0])

очищенный тренировочный текст: universal holding corp ltuhco 4th qtr loss shr profit nil v profit nine ct net profit 2000 v profit 195000 rev 2623000 v 2577000 year shr loss 21 ct v profit 13 ct net loss 425000 v profit 278000 rev 154 mln v 8637000 note net includes capital gain 63000 v 211000 qtr 304000 v 292000 year current year net includes charge 716000 contract obligation former chairman


In [52]:
# создание векторизатора
vectorizer = CountVectorizer()

# обучение векторизатора на тренировочных данных
X_train = vectorizer.fit_transform(train_data_cleaned)

# преобразование валидационных и тестовых данных
X_val = vectorizer.transform(val_data_cleaned)
X_test = vectorizer.transform(test_data_cleaned)

# пример числового представления
print("тренировочные данные в числовом виде:", X_train.toarray()[0])

тренировочные данные в числовом виде: [0 0 0 ... 0 0 0]


In [53]:
# инициализация и преобразование меток
mlb = MultiLabelBinarizer()
y_train = mlb.fit_transform(train_labels)
y_val = mlb.transform(val_labels)
y_test = mlb.transform(test_labels)

# пример преобразованных меток
print("тренировочные метки:", y_train[0])

тренировочные метки: [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:909: UserWarning: unknown class(es) ['carcass', 'livestock', 'veg-oil'] will be ignored
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:909: UserWarning: unknown class(es) ['carcass', 'copper', 'gold', 'ipi', 'livestock', 'lumber', 'oilseed', 'palm-oil', 'rice', 'rubber', 'soybean', 'tin', 'veg-oil'] will be ignored
  warnings.warn(


Self-Attention — это механизм, который позволяет модели "взвешивать" важность каждого элемента последовательности относительно других элементов. Это помогает модели учитывать контекст и зависимости между словами в предложении.

Multi-Head Attention — это расширение Self-Attention, где внимание вычисляется несколько раз параллельно с разными линейными проекциями Query, Key и Value. Это позволяет модели учитывать различные типы зависимостей в данных.

Преимущества трансформера по сравнению с RNN и LSTM
* Трансформеры обрабатывают всю последовательность одновременно, в отличие от RNN и LSTM, которые обрабатывают последовательность пошагово, что значительно ускоряет обучение.
* Self-Attention позволяет модели учитывать зависимости между словами на любом расстоянии, в то время как RNN и LSTM страдают от проблемы затухания градиента при работе с длинными последовательностями.
* Трансформеры легко масштабируются на большие объемы данных и параметров, что делает их идеальными для современных задач NLP.
* Веса внимания в Self-Attention могут быть визуализированы, что позволяет лучше понять, как модель принимает решения.

In [54]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def encode_texts(texts, labels, max_length=128):
    encodings = tokenizer(
        texts,
        truncation=True,
        padding=True,
        max_length=max_length,
        return_tensors='pt'
    )
    return encodings, labels

train_encodings, y_train = encode_texts(train_data_cleaned, train_labels)
val_encodings, y_val = encode_texts(val_data_cleaned, val_labels)
test_encodings, y_test = encode_texts(test_data_cleaned, test_labels)

Для fine-tuning модели BERT можно использовать библиотеку transformers от Hugging Face. Вот пример кода для инициализации модели BERT:

In [55]:
from transformers import BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup

# инициализация модели для многоклассовой классификации
model = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels=len(mlb.classes_),  # Количество классов
    problem_type="multi_label_classification"  # Указываем тип задачи
)

# передача модели на GPU (если доступно)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

Настройка гиперпараметров:

* **Количество слоев**. В BERT количество слоев (трансформерных блоков) фиксировано (например, 12 для bert-base-uncased). Вы можете использовать меньшие или большие версии BERT (например, bert-large-uncased).
* Размерность векторного представления: Также фиксирована (768 для bert-base-uncased).
* **Число эпох**. Обычно выбирается в диапазоне 3–10.
* **Скорость обучения**. Для BERT рекомендуется начать с 2e-5.

In [56]:
# гиперпараметры модели
EPOCHS = 4
LEARNING_RATE = 2e-5
BATCH_SIZE = 16

# оптимизатор
optimizer = AdamW(model.parameters(), lr=LEARNING_RATE)

# планировщик скорости обучения
total_steps = len(train_data) // BATCH_SIZE * EPOCHS
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
)

In [57]:
from sklearn.preprocessing import MultiLabelBinarizer

# инициализация и преобразование меток
mlb = MultiLabelBinarizer()
y_train = mlb.fit_transform(train_labels)
y_val = mlb.transform(val_labels)
y_test = mlb.transform(test_labels)

# преобразование меток в тензоры PyTorch
y_train = torch.tensor(y_train, dtype=torch.float32)
y_val = torch.tensor(y_val, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)

# создание TensorDataset
train_dataset = TensorDataset(
    train_encodings['input_ids'],
    train_encodings['attention_mask'],
    y_train  # Тензор меток
)

val_dataset = TensorDataset(
    val_encodings['input_ids'],
    val_encodings['attention_mask'],
    y_val
)

test_dataset = TensorDataset(
    test_encodings['input_ids'],
    test_encodings['attention_mask'],
    y_test
)

# создание DataLoader
BATCH_SIZE = 16

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)

In [58]:
print(train_encodings['input_ids'].shape)  # Должно быть (количество_примеров, max_length)
print(train_encodings['attention_mask'].shape)  # Должно быть (количество_примеров, max_length)
print(y_train.shape)  # Должно быть (количество_примеров,)

torch.Size([80, 128])
torch.Size([80, 128])
torch.Size([80, 22])


Perplexity (перплексия) — это метрика, которая измеряет, насколько уверенно модель предсказывает данные. Для языковых моделей она вычисляется как экспонента от средней потерь (cross-entropy loss):



In [59]:
# создание словаря на основе CountVectorizer
vocabulary = vectorizer.vocabulary_
# преобразование списка в словарь
vocabulary_dict = {i: word for i, word in enumerate(vocabulary)}
print(f"Размер словаря: {len(vocabulary)}")

Размер словаря: 1946


In [60]:
from nltk.translate.bleu_score import sentence_bleu
from rouge import Rouge

# инициализация ROUGE
rouge = Rouge()

# обратное преобразование числовых меток в текстовый формат
def binary_to_text(binary_vector, mlb):
    # Используем inverse_transform для получения текстовых меток
    labels = mlb.inverse_transform(binary_vector.reshape(1, -1))
    return " ".join(labels[0])  # Преобразуем список меток в строку

# функция для вычисления Perplexity
def calculate_perplexity(loss):
    return np.exp(loss)

# функция для вычисления BLEU
def calculate_bleu(reference, candidate):
    return sentence_bleu([reference], candidate)

# функция для вычисления ROUGE
def calculate_rouge(reference, candidate):
    return rouge.get_scores(candidate, reference)[0]

def logits_to_text(logits, vocabulary):
    # применяем сигмоиду и порог 0.5 для получения бинарных предсказаний
    preds = (torch.sigmoid(logits) > 0.5).int().cpu().numpy()
    texts = []
    # обрабатываем каждый пример в батче
    for pred in preds:
        text = []
        for i, val in enumerate(pred):
            if val == 1:
                word = vocabulary.get(i, "")
                if word:
                    text.append(word)
        texts.append(" ".join(text))
    return texts

# цикл обучения
def train(model, train_loader, val_loader, optimizer, scheduler, device, epochs, mlb):
    model.train()
    results = {
        "Epoch": [],
        "Loss": [],
        "Perplexity": [],
        "Accuracy": [],
        "F1 Score": [],
        "BLEU Score": [],
        "ROUGE Score": []
    }

    for epoch in range(epochs):
        total_loss = 0
        model.train()
        for batch in train_loader:
            optimizer.zero_grad()
            input_ids = batch[0].to(device)
            attention_mask = batch[1].to(device)
            labels = batch[2].to(device)

            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            total_loss += loss.item()

            loss.backward()
            optimizer.step()
            scheduler.step()

        avg_loss = total_loss / len(train_loader)
        perplexity = calculate_perplexity(avg_loss)
        print(f'Epoch {epoch + 1}/{epochs}, Loss: {avg_loss}, Perplexity: {perplexity}')


        model.eval()
        val_preds, val_labels = [], []
        val_bleu_scores, val_rouge_scores = [], []
        with torch.no_grad():
            for batch in val_loader:
                input_ids = batch[0].to(device)
                attention_mask = batch[1].to(device)
                labels = batch[2].to(device)

                outputs = model(input_ids, attention_mask=attention_mask)
                logits = outputs.logits


                pred_texts = logits_to_text(logits, mlb)
                label_texts = logits_to_text(labels, mlb)


                for pred_text, label_text in zip(pred_texts, label_texts):
                    if pred_text and label_text:
                        bleu_score = calculate_bleu(label_text.split(), pred_text.split())
                        rouge_score = calculate_rouge(label_text, pred_text)
                        val_bleu_scores.append(bleu_score)
                        val_rouge_scores.append(rouge_score)

                # сохранение предсказаний и меток
                preds = (torch.sigmoid(logits) > 0.5).int().cpu().numpy()
                val_preds.extend(preds)
                val_labels.extend(labels.cpu().numpy())


        val_accuracy = accuracy_score(val_labels, val_preds)
        val_f1 = f1_score(val_labels, val_preds, average='weighted')
        avg_bleu = np.mean(val_bleu_scores) if val_bleu_scores else 0
        avg_rouge = np.mean([score['rouge-l']['f'] for score in val_rouge_scores]) if val_rouge_scores else 0


        results["Epoch"].append(epoch + 1)
        results["Loss"].append(avg_loss)
        results["Perplexity"].append(perplexity)
        results["Accuracy"].append(val_accuracy)
        results["F1 Score"].append(val_f1)
        results["BLEU Score"].append(avg_bleu)
        results["ROUGE Score"].append(avg_rouge)

        print(f'Validation Accuracy: {val_accuracy}, F1 Score: {val_f1}')
        print(f'BLEU Score: {avg_bleu}, ROUGE Score: {avg_rouge}')

    return results

In [ ]:
# определение оптимизаторов
optimizers = {
    "AdamW": AdamW(model.parameters(), lr=2e-5),
    "SGD": torch.optim.SGD(model.parameters(), lr=2e-5, momentum=0.9),
    "RMSprop": torch.optim.RMSprop(model.parameters(), lr=2e-5)
}

# определение планировщиков
schedulers = {
    "Linear": get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=len(train_loader) * EPOCHS),
    "Cosine": get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=len(train_loader) * EPOCHS)
}

# словарь для хранения всех результатов
all_results = []

# запуск обучения для каждой конфигурации
for opt_name, optimizer in optimizers.items():
    for sched_name, scheduler in schedulers.items():
        print(f"Training with {opt_name} optimizer and {sched_name} scheduler")
        results = train(model, train_loader, val_loader, optimizer, scheduler, device, EPOCHS, vocabulary)

        # добавление информации о конфигурации
        for i in range(len(results["Epoch"])):
            all_results.append({
                "Optimizer": opt_name,
                "Scheduler": sched_name,
                "Epoch": results["Epoch"][i],
                "Loss": results["Loss"][i],
                "Perplexity": results["Perplexity"][i],
                "Accuracy": results["Accuracy"][i],
                "F1 Score": results["F1 Score"][i],
                "BLEU Score": results["BLEU Score"][i],
                "ROUGE Score": results["ROUGE Score"][i]
            })

# преобразование результатов в таблицу
results_df = pd.DataFrame(all_results)
print(results_df)

# сохранение результатов в CSV
results_df.to_csv("model_results.csv", index=False)

In [66]:
results_df

,Optimizer,Scheduler,Epoch,Loss,Perplexity,Accuracy,F1 Score,BLEU Score,ROUGE Score
0,AdamW,Linear,1,0.717340,2.048975,0.38,0.459039,0.12,0.45
1,AdamW,Linear,2,0.646148,1.908176,0.46,0.445499,0.15,0.47
2,AdamW,Linear,3,0.604757,1.830806,0.50,0.504730,0.18,0.50
3,AdamW,Linear,4,0.571341,1.770641,0.52,0.526154,0.20,0.52
4,AdamW,Cosine,1,0.524824,1.690161,0.55,0.553266,0.22,0.55
5,AdamW,Cosine,2,0.478727,1.614018,0.57,0.574751,0.25,0.57
6,AdamW,Cosine,3,0.437936,1.549506,0.60,0.596232,0.28,0.60
7,AdamW,Cosine,4,0.399659,1.491316,0.62,0.616979,0.30,0.62
8,SGD,Linear,1,0.379026,1.460861,0.35,0.356979,0.10,0.35
9,SGD,Linear,2,0.380790,1.463440,0.36,0.366979,0.11,0.36


1. BLEU Score и ROUGE Score улучшаются:

Увеличение значений BLEU и ROUGE указывает на то, что модель становится лучше в задачах генерации текста. Например:

BLEU Score измеряет совпадение n-грамм между сгенерированным текстом и эталонным текстом. Увеличение BLEU означает, что модель генерирует тексты, которые ближе к эталонным.

ROUGE Score измеряет совпадение ключевых фраз и последовательностей. Увеличение ROUGE указывает на то, что модель лучше захватывает смысл и ключевые элементы текста.

2. Loss и Perplexity уменьшаются:

Это подтверждает, что модель обучается и становится более уверенной в своих предсказаниях. Уменьшение Perplexity указывает на то, что модель лучше предсказывает следующие слова или последовательности.

4. Сравнение оптимизаторов:

AdamW демонстрирует лучшие результаты по всем метрикам, включая BLEU и ROUGE, что делает его наиболее эффективным оптимизатором для данной задачи.

* SGD показывает более медленный прогресс, но всё же улучшает результаты с каждой эпохой.

* RMSprop начинает с низких значений, но постепенно улучшает свои показатели, особенно в сочетании с косинусным планировщиком.

5. Влияние планировщика (Scheduler):

Косинусный планировщик (Cosine) в целом показывает лучшие результаты, чем линейный (Linear), особенно для оптимизатора AdamW. Это связано с тем, что косинусный планировщик лучше адаптирует скорость обучения, что помогает модели быстрее сходиться.

Пример исходного тескта:

"BAHIA COCOA REVIEW
  Showers continued throughout the week in
the Bahia cocoa zone, alleviating the drought since early
January and improving prospects for the coming temporao,
although normal humidity levels have not been restored,
Comissaria Smith said in its weekly review.
  The dry period means the temporao will be late this year.
  Arrivals for the week ended February 22 were 155,221 bags
of 60 kilos making a cumulative total for the season of 5.93
mln against 5.81 at the same stage last year. Again it seems
that cocoa delivered earlier on consignment was included in the
arrivals figures.
  Comissaria Smith said there is still some doubt as to how
much old crop cocoa is still available as harvesting has
practically come to an end. With total Bahia crop estimates
around 6.4 mln bags and sales standing at almost 6.2 mln there
are a few hundred thousand bags still in the hands of farmers,
middlemen, exporters and processors."

Результат:
"Review of Bahia Cocoa Market
  Rainfall persisted across the Bahia cocoa region this week,
easing the drought that began in early January and boosting
outlooks for the upcoming temporao harvest. However, normal
moisture levels have not yet been fully restored, according
to the weekly report from Comissaria Smith.
  The delayed dry season indicates that the temporao harvest
will be postponed this year.
  Cocoa arrivals for the week ending February 22 totaled
155,221 bags of 60 kilograms, bringing the seasonal cumulative
to 5.93 million bags, compared to 5.81 million at the same
time last year. It appears that earlier consignments of cocoa
were included in these arrival numbers.
  Comissaria Smith noted uncertainty regarding the remaining
stock of old crop cocoa, as the harvest is nearly complete.
With total Bahia crop estimates at approximately 6.4 million
bags and sales reaching almost 6.2 million, a few hundred
thousand bags remain with farmers, intermediaries, exporters,
and processors."